# 구매 후기 긍정 / 부정 예측하기

## 데이터 불러오기

In [1]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
from sklearn.model_selection import train_test_split

In [2]:
# 경고창 무시
import warnings
warnings.filterwarnings(action='ignore')

In [3]:
review = pd.read_csv('../data_list/review_202205161607.csv', encoding='utf-8-sig')
review

,review_idx,product_idx,user_id,review_content,review_img,review_score,review_rgstdate
0,1,1,admin,색감이 곱네요,good!.png,4.5,2022-05-13 15:10:51
1,3,1,aaaaaa1234,색이 오묘한게 정말 예뻐요 여름에 진짜 잘 입을 것 같아용 케키케,good!.png,5.0,2022-05-13 15:20:53
2,4,1,use06048,우선 옷은 되게 얇아요! 그리고 제가 키가 큰 편이라 너무 짧고 작을까봐 걱정했는데...,good!.png,4.5,2022-05-13 15:20:53
3,5,1,qwer,색감이 너무 마음에 들어요! 사이즈도 괜찮고 디자인도 예뻐요~,good!.png,4.0,2022-05-13 15:20:53
4,6,1,sdf,받고 나서 깜짝 놀랐네요 벽돌색이라고 하고 파세요 상품 사진에 나온 것보다 한참 어...,BAD.png,0.5,2022-05-13 15:20:53
5,7,1,use06048,예뻐요 짱짱 그냥 고민 없이 사세요 늦게 산 걸 후회하는 중..,well...png,3.5,2022-05-13 15:20:53
6,8,1,aaaaaa1234,배송은 빠르고 이런 스타일 좋아하는 사람은 좋아하겠고 레이어드 해서 입힐 생각,well...png,4.0,2022-05-13 15:20:53
7,9,1,qwer,생각보다 더 얇고 작아요 ㅎㅎ 편하게 입기 좋을거 같습니다,good!.png,3.0,2022-05-13 15:20:53
8,10,1,use06048,키 골격 큰편이어서 작을까봐 걱정했는데 생각보다 괜찮네영 여름에 롱스커트랑 닥마에 ...,well...png,4.5,2022-05-13 15:20:53
9,11,1,sdf,옷만 보면 이쁨 바지도 청이나 어두운 계열 잘 어울릴 거 같아여,good!.png,3.5,2022-05-13 15:20:53


## 데이터 정제

In [4]:
# 한글과 공백을 제외하고 모두 제거
review['review_content'] = review['review_content'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
review['review_content'].replace('', np.nan, inplace=True)
print(review.isnull().sum())

review_idx         0
product_idx        0
user_id            0
review_content     0
review_img         0
review_score       0
review_rgstdate    0
dtype: int64


In [5]:
review['review_content'][:3]

0                                              색감이 곱네요
1                 색이 오묘한게 정말 예뻐요 여름에 진짜 잘 입을 것 같아용 케키케
2    우선 옷은 되게 얇아요 그리고 제가 키가 큰 편이라 너무 짧고 작을까봐 걱정했는데 ...
Name: review_content, dtype: object

## 평점이 3보다 크면 긍정(1), 그렇지 않다면 부정(0)으로 분류해서 label이라는 컬럼 생성

In [6]:
review['label'] = np.select([review['review_score'] > 3], [1], default=0)
review[:5]

,review_idx,product_idx,user_id,review_content,review_img,review_score,review_rgstdate,label
0,1,1,admin,색감이 곱네요,good!.png,4.5,2022-05-13 15:10:51,1
1,3,1,aaaaaa1234,색이 오묘한게 정말 예뻐요 여름에 진짜 잘 입을 것 같아용 케키케,good!.png,5.0,2022-05-13 15:20:53,1
2,4,1,use06048,우선 옷은 되게 얇아요 그리고 제가 키가 큰 편이라 너무 짧고 작을까봐 걱정했는데 ...,good!.png,4.5,2022-05-13 15:20:53,1
3,5,1,qwer,색감이 너무 마음에 들어요 사이즈도 괜찮고 디자인도 예뻐요,good!.png,4.0,2022-05-13 15:20:53,1
4,6,1,sdf,받고 나서 깜짝 놀랐네요 벽돌색이라고 하고 파세요 상품 사진에 나온 것보다 한참 어...,BAD.png,0.5,2022-05-13 15:20:53,0


In [7]:
review_data = review[['review_content', 'label']]
review_data[:3]

,review_content,label
0,색감이 곱네요,1
1,색이 오묘한게 정말 예뻐요 여름에 진짜 잘 입을 것 같아용 케키케,1
2,우선 옷은 되게 얇아요 그리고 제가 키가 큰 편이라 너무 짧고 작을까봐 걱정했는데 ...,1


In [8]:
X_train, X_test, y_train, y_test = train_test_split(
            review['review_content'],
            review['label'],
            test_size=0.2,
            random_state=42
)

## RANKS NL에서 제공하는 한국어 불용어 사전 사용 & vectorize => choice(Counter, Tf-idf) : fit(train) —> transform(train,test)

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [10]:
stopwords = pd.read_csv("https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/korean_stopwords.txt").values.tolist()
stopwords = sum(stopwords, [])
stopwords[:10]

['휴', '아이구', '아이쿠', '아이고', '어', '나', '우리', '저희', '따라', '의해']

In [11]:
tfidf = TfidfVectorizer(lowercase = True, stop_words=stopwords)

In [12]:
tfidf.fit(X_train)

TfidfVectorizer(stop_words=['휴', '아이구', '아이쿠', '아이고', '어', '나', '우리', '저희',
                            '따라', '의해', '을', '를', '에', '의', '가', '으로', '로',
                            '에게', '뿐이다', '의거하여', '근거하여', '입각하여', '기준으로', '예하면',
                            '예를 들면', '예를 들자면', '저', '소인', '소생', '저희', ...])

In [13]:
X_train =  tfidf.transform(X_train)

In [14]:
X_test= tfidf.transform(X_test)

In [15]:
print(X_train.shape, X_test.shape)

(8, 91) (3, 91)


In [16]:
tfidf.inverse_transform(X_train[:1])

[array(['의심스럽네요', '알바가', '아닌가', '별로에요', '댓글들이'], dtype='<U7')]

In [17]:
tfidf.inverse_transform(X_test[:1])

[array(['예뻐요', '사세요', '그냥'], dtype='<U7')]

## ml model fit(train) => predict(test) => accuracy check

In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [19]:
SA_lr = LogisticRegression(random_state = 0)

In [20]:
SA_lr.fit(X_train, y_train)

LogisticRegression(random_state=0)

In [21]:
test_predict = SA_lr.predict(X_test)

In [22]:
print('긍정 / 부정 분석 정확도 : ', round(accuracy_score(y_test, test_predict), 5))

긍정 / 부정 분석 정확도 :  1.0


## one review => predict!!

In [23]:
nr = review['review_content'][6]

In [24]:
nr = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", nr)
nr

'배송은 빠르고 이런 스타일 좋아하는 사람은 좋아하겠고 레이어드 해서 입힐 생각'

In [25]:
nr_tfidf = tfidf.transform([nr])
nr_tfidf

<1x91 sparse matrix of type '<class 'numpy.float64'>'
	with 10 stored elements in Compressed Sparse Row format>

In [26]:
nr_predict = SA_lr.predict(nr_tfidf)
nr_predict, nr_predict[0]

(array([1]), 1)

In [27]:
if(nr_predict[0] == 1):
    print(nr, '==> 긍정적인 후기입니다')
else:
    print(nr, '==> 부정적인 후기입니다')

배송은 빠르고 이런 스타일 좋아하는 사람은 좋아하겠고 레이어드 해서 입힐 생각 ==> 긍정적인 후기입니다
